# worksheet 9: Pandas DataFrames (contd)


In [ ]:
import pandas as pd
import numpy as np

### Create `gene_expression_df`

In [ ]:
expression_dict = {
    'BRCA1': [2, 3],
    'BRCA2': [4, 5],
    'SMAD2': [4, 5],
    'TTN': [5, 6]
}

In [ ]:
gene_expression_df = pd.DataFrame(
    expression_dict,
    index=['time.0', 'time.1']
)

In [ ]:
gene_expression_df

In [ ]:
gene_expression_df = gene_expression_df.T

In [ ]:
gene_expression_df

### Looping and aggregation
- apply functions on different axes
- std functions (log, exp)
- custom functions

In [ ]:
gene_expression_df.apply(np.log)

In [ ]:
gene_expression_df.apply(np.exp)

### custom functions with apply

In [ ]:
gene_expression_df.apply(
    lambda x: x + 2, axis=1
)

In [ ]:
gene_expression_df.apply(
    lambda x: x + 2, axis=0
)

In [ ]:
gene_expression_df.apply(
    lambda x: print(x), axis=0
)

In [ ]:
gene_expression_df.apply(
    lambda x: print(x), axis=1
)

### sum across the columns

In [ ]:
gene_expression_df.apply(
    lambda x: x['time.0'] + x['time.1'], axis=1
)

In [ ]:
gene_expression_df.apply(sum, axis=1)

In [ ]:
gene_expression_df.sum(axis=1)

### sum down the rows

In [ ]:
gene_expression_df.sum(axis=0)

### Q: How is this different from running apply on a series?

In [ ]:
gene_expression_df['time.0']

In [ ]:
gene_expression_df['time.0'].apply(np.log)

In [ ]:
gene_expression_df['time.0'].apply(lambda x: print(x))

In [ ]:
gene_expression_df['time.0'].sum()

In [ ]:
gene_expression_df['time.0'].sum(axis=0)

### Q: Will the following work?


In [ ]:
# gene_expression_df['time.0'].apply(lambda x: print(x), axis=1)

### Q: How about the following?

In [ ]:
# gene_expression_df['time.0'].sum(axis=1)

## Groupby, melting, stacking
- pd.concat (stacking)

In [ ]:
gene_expression_df

In [ ]:
gene_expression_df['gene_type'] = ['coding'] * 4

In [ ]:
gene_expression_df['gene_len_bp'] = [2000, 4000, 4000, 5000]

In [ ]:
gene_expression_df

In [ ]:
non_coding_df = pd.DataFrame({
    'mirna123': [np.nan, np.nan, 'non-coding', 22],
    'mirna456': [np.nan, np.nan, 'non-coding', 25],
    'lncRNA1': [np.nan, np.nan, 'non-coding', 500],
}).T

In [ ]:
non_coding_df.columns=['time.0', 'time.1','gene_type', 'gene_len_bp']

In [ ]:
non_coding_df

In [ ]:
groupby_df = pd.concat([gene_expression_df, non_coding_df])

In [ ]:
groupby_df

### Q: what is the average gene length in bp by the type of gene (coding, or non-coding)?

In [ ]:
g = groupby_df.groupby('gene_type')

In [ ]:
g.groups

In [ ]:
groupby_df.groupby('gene_type')['gene_len_bp'].mean()

In [ ]:
groupby_df.groupby('gene_type').agg({'gene_len_bp': 'mean'})

In [ ]:
groupby_df.groupby('gene_type').agg(
    {'gene_len_bp': 'mean',
    'time.0': 'mean'
    }
)

In [ ]:
groupby_df.groupby('gene_type').agg(
    {'gene_len_bp': ['mean', 'min', 'max'],
    'time.0': ['mean', 'min', 'max'],
    }
)

In [ ]:
groupby_df.groupby('gene_type').agg(
    mean_gene_len_bp=('gene_len_bp','mean'),
    min_gene_len_bp=('gene_len_bp', 'min'),
    max_gene_len_bp=('gene_len_bp', 'max'),
    mean_time_0=('time.0','mean'),
    min_time_0=('time.0', 'min'),
    max_time_0=('time.0', 'max')
)

In [ ]:
res = groupby_df.groupby('gene_type').agg(
    mean_gene_len_bp=('gene_len_bp','mean'),
    min_gene_len_bp=('gene_len_bp', 'min'),
    max_gene_len_bp=('gene_len_bp', 'max'),
    mean_time_0=('time.0','mean'),
    min_time_0=('time.0', 'min'),
    max_time_0=('time.0', 'max')
)

In [ ]:
type(res)

In [ ]:
res.at['coding', 'mean_gene_len_bp']

In [ ]:
res.at['non-coding', 'mean_gene_len_bp']

### Q: what is the average gene expression at time point 0 by the type of gene (coding, or non-coding)?

In [ ]:
groupby_df

In [ ]:
groupby_df.groupby('gene_type')['time.0'].mean()

### pivot table
- alternative to groupby
- index = keys to group by on the index
- values = columns to aggregate

In [ ]:
groupby_df.groupby('gene_type')['gene_len_bp'].mean()

In [ ]:
groupby_df.pivot_table(
    values='gene_len_bp', 
    index='gene_type',
    aggfunc='mean'
)

In [ ]:
groupby_df.groupby('gene_type')['time.0'].mean()

In [ ]:
groupby_df.pivot_table(
    values='time.0',
    index='gene_type',
    aggfunc='mean'
)

### Groupby multiple columns
- can you add a new column to groupby_df called 'gt_1000' that outputs 'yes' if gene length > 1000bp, and 'no' if not?

In [ ]:
x = groupby_df['gene_len_bp'] > 1000

In [ ]:
groupby_df['gt_1000'] = ['yes' if i else 'no' for i in x]

In [ ]:
groupby_df

### Q: can you do the above with a lambda function?
- add a new column called `gt_1000_v2` to the df, with the values computed using a lambda function

In [ ]:
groupby_df['gt_1000_v2'] = groupby_df['gene_len_bp'].apply(
    lambda x: 'yes' if x > 1000 else 'no'
)

In [ ]:
groupby_df['gt_1000_v2']

In [ ]:
groupby_df.groupby(['gene_type', 'gt_1000'])['gene_len_bp'].mean()

In [ ]:
# same as above, with pivot_table
groupby_df.pivot_table(
    index=['gene_type', 'gt_1000'],
    values='gene_len_bp',
    aggfunc='mean'
)

### Melting

In [ ]:
groupby_df

### Q: can you reshape the above to:
- have only one time column
- still represent time.0 and time.1 as a variable
- retain gene name, gene type and gene_len_bp cols

In [ ]:
groupby_df.reset_index()

In [ ]:
df_to_melt = groupby_df.reset_index().rename(columns={'index':'gene_name'})

In [ ]:
df_to_melt

In [ ]:
df_to_melt.melt(
    id_vars=['gene_name','gene_type', 'gene_len_bp'],
    value_vars=['time.0', 'time.1']
)

## Joins
- pandas merge function, SQL style joining operations
- https://pandas.pydata.org/docs/user_guide/merging.html

In [ ]:
d1 = {
    'gene_name': ['BRCA1', 'KRAS', 'HRAS', 'TP53'],
    'gene_length': [2, 4, 4, 6]
}
left_df = pd.DataFrame(d1)

In [ ]:
left_df

In [ ]:
d2 = {
    'gene_name': ['BRCA1', 'KRAS', 'HRAS', 'TP53'],
    'num_transcripts': [4, 6, 3, 7]
}
right_df = pd.DataFrame(d2)

In [ ]:
right_df

#### simple join on 'gene_name'

In [ ]:
pd.merge(left=left_df, right=right_df, on='gene_name')

In [ ]:
pd.merge(
    left=left_df, 
    right=right_df, 
    on='gene_name', 
    indicator=True
)

### left join
- use keys from left frame only

In [ ]:
d1 = {
    'gene_name': ['BRCA1', 'HRAS', 'TP53', 'NRAS'],
    'gene_length': [2, 4, 6, 8]
}
left_df = pd.DataFrame(d1)

In [ ]:
d2 = {
    'gene_name': ['BRCA1', 'KRAS', 'HRAS', 'TP53'],
    'num_transcripts': [4, 6, 3, 7]
}
right_df = pd.DataFrame(d2)

In [ ]:
pd.merge(
    left=left_df, 
    right=right_df, 
    on='gene_name', 
    how='left',
    indicator=True
)

### right join
- use keys from right frame only

In [ ]:
left_df

In [ ]:
right_df

In [ ]:
pd.merge(
    left=left_df, 
    right=right_df, 
    on='gene_name', 
    how='right',
    indicator=True
)

### inner join
- intersection of keys from both frames

In [ ]:
pd.merge(
    left=left_df, 
    right=right_df, 
    on='gene_name', 
    how='inner',
    indicator=True
)

### outer join
- union of keys from both frames

In [ ]:
pd.merge(
    left=left_df, 
    right=right_df, 
    on='gene_name', 
    how='outer',
    indicator=True
)

# Collaborative Exercises

## Exercise 1
- Check out the uploaded CSV file on github: `genes_transcripts.csv` and read it into a pandas dataframe
-  What is the shape of the dataframe?
- How many unique genes are there in the `gene_name` column? Can you list them? How about the same for the `biotype` column? Hint: see if you can use https://pandas.pydata.org/docs/reference/api/pandas.Series.nunique.html and https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html
- What is the average transcript length by gene for each biotype?
- What are the min and max transcript lengths by gene for each biotype?
- Can you also do the above two exercises using pivot_table?
- What percentage of transcripts are protein_coding for each gene?


## Exercise 2
- You will create two pandas dataframes of your own, from your specific field, to demonstrate how joins work in pandas
- Create two dataframes from two dictionaries, each with shape atleast (4, 3)
- Demonstrate inner join, left join, right join and outer join
- Make sure to include the appropriate data to demo all types of joins